# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**: Jaime Enrique Galindo Villegas

**Professor**: Pablo Camarillo Ramirez

# Introducción: Análisis de Transacciones en Mercado Libre
La aplicación web y móvil de Mercado libre procesa millones de transacciones diarias que pueden ser pedidos de artículos, suscripciones, pagos, etc. En el 2023 la plataforma logró ventas de 14,500 millones de dólares, con picos de demanda en momentos significativos del año como en el "Buen Fin" o el "Hot Sale" en los que sus ventas se disparan, en el 2024 llegaron a picos de más de 70% más ventas en el buen fin respecto a años anteriores.  

Este volumen masivo de datos, generado en multiples formatos presenta un desafio. Para abordarlo implementaré un pipeline de datos que procesa la información de órdenes de compra, con el objetivo de transformar los datos crudos en un formato limpio y estructurado que facilite posteriores analisis.

# Dataset: Órdenes de Compra

## Modelo de Datos
Para este problema, el **modelo de datos** más adecuado es el de **documento**.

Esto debido a que, según el diseño de pipeline que cree anteriormente, una orden de compra de ML es una entidad (JSON) que contiene toda su información de manera autocontenida, el usuario, monto, detalles de envio y articulos comprados. Por esto el modelo de documento es ideal al permitir presentar esta estructura jerárquica.

Esto es diferente de un modelo **relacional** ya que tendria que normalizar y dividir la información en multiples tablas como `Ordenes`, `Usuarios`, `Productos`, etc. Requeriria operaciones más complejas y, debido a la necesidad de procesar millones de transacciones, es mejor un modelo de **documento** que es más naturalmente compatible


## Generación del Dataset y Esquema
Para simular el flujo de datos de ML, se generará un archivo json (`orders.json`) usando la libreria Faker, en el que cada linea representará un objeto JSON de una orden de compra. Así simularé un sistema de ingesta en el que se recibirian los datos en tiempo real.

El **esquema** de cada objeto JSON generado será el siguiente:

- **order_id**: `String` - Identificador único de la orden.
- **timestamp**: `String` - Fecha y hora de la transacción en formato ISO.
- **user**: `Struct` - Objeto usuario.
  - **user_id**: `String` - Identificador del usuario.
  - **region**: `String` - Región de la compra (ej. "GDL").
  - **payment_method**: `String` - Método de pago utilizado (ej. "MP").
- **items**: `Array` - Lista de artículos comprados.
  - **item_id**: `String` - ID del producto.
  - **title**: `String` - Nombre del producto.
  - **category**: `String` - Categoría del producto.
  - **quantity**: `Integer` - Cantidad de unidades.
  - **unit_price**: `Double` - Precio original por unidad.
  - **final_price**: `Double` - Precio final por unidad con descuento.
  - **discount_applied**: `Boolean` - Indica si se aplicó descuento.
- **total_amount**: `Double` - Monto total de la orden.
- **shipping**: `Struct` - Información de envío.
  - **logistics_provider**: `String` - Empresa de paquetería.
  - **warehouse_origin**: `String` - Almacén de origen.
  - **estimated_delivery**: `String` - Fecha estimada de entrega.
  - **tracking_id**: `String` - Número de seguimiento.

In [1]:
from jaime_galindo.order_generator import OrderGenerator
import json

# Generar 1000 órdenes y guardarlas en un archivo JSON
# cada JSON esté en una nueva línea para Spark
with open("./data/batch_processing/orders.json", "w") as f:
	for i in range(1000):
		f.write(json.dumps(OrderGenerator.create_random_order(i)) + '\n')

print("Archivo orders.json generado con 1000 órdenes.")

Archivo orders.json generado con 1000 órdenes.


# Transformations and Actions

### Iniciar sesión de spark

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MercadoLibreTransformations") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/25 05:25:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Leer los datos con esquema

In [3]:
from pyspark.sql.types import ArrayType # type: ignore
from jaime_galindo.spark_utils import SparkUtils

# Esquemas básicos de cada entidad
user_schema = SparkUtils.generate_schema([
	("user_id", "string"),
	("region", "string"),
	("payment_method", "string")
])

item_schema = SparkUtils.generate_schema([
	("item_id", "string"),
	("title", "string"),
	("category", "string"),
	("quantity", "integer"),
	("unit_price", "double"),
	("final_price", "double"),
	("discount_applied", "boolean")
])

shipping_schema = SparkUtils.generate_schema([
	("logistics_provider", "string"),
	("warehouse_origin", "string"),
	("estimated_delivery", "string"),
	("tracking_id", "string")
])


# Equema principal de la orden
order_schema = SparkUtils.generate_schema([
	("order_id", "string"),
	("timestamp", "string"),
	("user", user_schema),
	("items", ArrayType(item_schema)),
	("total_amount", "double"),
	("shipping", shipping_schema)
])

In [4]:
# Lectura de datos
df_raw_orders = spark.read \
    .schema(order_schema) \
    .json("./data/batch_processing/orders.json")


In [5]:
# Transformaciones
from pyspark.sql.functions import col, explode 
# , year, month, dayofmonth, current_timestamp, split, lit

# hacer explode para obtener las columnas de cada elemento y filas para cada item
df_with_items = df_raw_orders.select(
   col("order_id"),
   col("timestamp"),
   col("user"),
   col("total_amount"),
   col("shipping"),

	explode("items").alias("item")
)

# "Aplanar" cada estructura compleja para tener solo los valores
df_flattened = df_with_items.select(
   col("order_id"),
   col("timestamp"),
   col("total_amount").alias("order_total_amount"),
   
	# Aplanar user
   col("user.user_id"),
   col("user.region"),
   col("user.payment_method"),
   
	# Aplanar shipping,
   col("shipping.logistics_provider"),
   col("shipping.warehouse_origin"),
   col("shipping.estimated_delivery"),
   col("shipping.tracking_id"),
   
	# Aplanar item,
	col("item.item_id"),
	col("item.title"),
	col("item.category"),
	col("item.quantity"),
	col("item.unit_price"),
	col("item.final_price"),
	col("item.discount_applied")

)


In [6]:
from pyspark.sql.functions import current_timestamp, year, month, dayofmonth
from pyspark.sql.types import TimestampType

# Enriquecimiento, crear nuevas columnas para agregar valor a los datos
df_enriched = df_flattened \
    .withColumn("order_timestamp", col("timestamp").cast(TimestampType())) \
    .withColumn("item_total_price", col("final_price") * col("quantity")) \
    .withColumn("item_total_discount", (col("unit_price") - col("final_price")) * col("quantity")) \
    .withColumn("processing_timestamp", current_timestamp()) 

# Particionado por fecha para optimizar la insersión más adelante
final_df = df_enriched.withColumn("year", year(col("order_timestamp"))) \
                      .withColumn("month", month(col("order_timestamp"))) \
                      .withColumn("day", dayofmonth(col("order_timestamp")))



# Persistence Data

# DAG